In [1]:
from funct import *

pygame 2.6.1 (SDL 2.28.4, Python 3.11.9)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [ ]:
def sentence_gen():
    '''creates a simple example sentnece using the word provided as well as words from ones libary based on your fluceny level of them
    - sort by word only above fluency level 3.    or something. 
    '''
    return 0



def give_me_a_word():
    '''Will open up a window and will give the next practice word for you to learn or to review pygame pop up? -- should call from most and 
    using the gen sentece pornounce and more
    '''

    return 0


def pick_next_word():
    '''will use an algo to start picking next words ect..'''

    return 0

In [5]:
import pygame

# Initialize Pygame
pygame.init()

# Set up display
WIDTH, HEIGHT = 500, 400
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Word Display with Buttons")

# Colors
WHITE = (255, 255, 255)
BLUE = (0, 122, 204)
DARK_BLUE = (0, 90, 150)
BLACK = (0, 0, 0)

# Font
font = pygame.font.Font(None, 50)
button_font = pygame.font.Font(None, 30)

# Word list
words = ["Example", "Python", "Code", "Pygame", "Button"]
word_index = 0

def get_current_word():
    return words[word_index]

# Button properties
next_button_rect = pygame.Rect(125, 250, 100, 50)
popup_button_rect = pygame.Rect(275, 250, 100, 50)

def show_popup():
    print("Popup function triggered!")

def next_word():
    global word_index
    word_index = (word_index + 1) % len(words)
    print(f"Next word displayed: {get_current_word()}")

# Main loop
running = True
while running:
    screen.fill(WHITE)
    
    # Draw the word in the center
    text_surface = font.render(get_current_word(), True, BLACK)
    text_rect = text_surface.get_rect(center=(WIDTH//2, 100))
    screen.blit(text_surface, text_rect)
    
    # Draw buttons
    pygame.draw.rect(screen, BLUE, next_button_rect)
    pygame.draw.rect(screen, BLUE, popup_button_rect)
    
    # Draw button labels
    next_text = button_font.render("Next", True, WHITE)
    popup_text = button_font.render("Popup", True, WHITE)
    screen.blit(next_text, next_button_rect.move(25, 15))
    screen.blit(popup_text, popup_button_rect.move(15, 15))
    
    # Event handling
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        elif event.type == pygame.MOUSEBUTTONDOWN:
            if next_button_rect.collidepoint(event.pos):
                next_word()
            elif popup_button_rect.collidepoint(event.pos):
                show_popup()
    
    pygame.display.flip()

pygame.quit()


Next word displayed: Python
Next word displayed: Code
Next word displayed: Pygame
Next word displayed: Button
Next word displayed: Example
Next word displayed: Python
Next word displayed: Code
Next word displayed: Pygame
Next word displayed: Button
Next word displayed: Example
Next word displayed: Python
Next word displayed: Code
Next word displayed: Pygame
Popup function triggered!
Popup function triggered!
Next word displayed: Button
Next word displayed: Example
Next word displayed: Python
Next word displayed: Code
Next word displayed: Pygame
Next word displayed: Button
